# Twitter API

Typical ways of collecting text and non-text data from the web are:
* APIs
* HTML scraping

Now let's retrieve some data from twitter and store to use it later in other notebooks.
Some of the data we can capture from twitter include:
1. Tweets
2. User information

In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext watermark
%watermark -v -m -p numpy,pandas,tweepy -g

import os
import csv
import random
import yaml
import tweepy as tw
from tqdm import tqdm, trange
from time import sleep
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import watermark

CPython 3.7.3
IPython 7.5.0

numpy 1.16.4
pandas 0.24.2
tweepy 3.7.0

compiler   : GCC 7.3.0
system     : Linux
release    : 5.0.0-20-generic
machine    : x86_64
processor  : x86_64
CPU cores  : 8
interpreter: 64bit
Git hash   : 4ffecf583a224874a147bea189021da33065398d


### Constants

Modify these values to update the config file, the query or the output file.

In [2]:
CONFIG_FILE = "./data/config/twitter_config.yml"
SEARCH_WORDS = "#film"
DATE_SINCE = "2019-06-27"
LIMIT = 1000
TWEETS_OUTPUT = "./data/raw/tweets.csv"
TWITER_USERS_FILE = './data/raw/twitter_users.csv'

### How to get Twitter Consumer key and Consumer secret key

1. Go to https://dev.twitter.com/apps/new and log in, if necessary
2. Apply for new Developer account
3. Supply the necessary required fields, accept the Terms Of Service, and solve the CAPTCHA.
4. Submit the form
5. Create new App
6. Go to the tab of Keys and tokens
6. Copy the consumer key (API key) and consumer secret from the screen into our application.

Load twitter keys from the yaml file. The file contains the following variables:
* access_token
* access_token_secret
* consumer_api_key
* consumer_api_secret_key

In [3]:
with open(CONFIG_FILE, 'r') as f:
    twitter_keys = yaml.safe_load(f)

Load twitter tokens to authenticate the access:

In [4]:
auth = tw.OAuthHandler(twitter_keys['consumer_api_key'], twitter_keys['consumer_api_secret_key'])
auth.set_access_token(twitter_keys['access_token'], twitter_keys['access_token_secret'])

api = tw.API(auth, api_root='/1.1', wait_on_rate_limit=True)

Get public tweets by using a specific search and limited to a number of tweets. <br>
Note: The Twitter Search API returns at max 3200 of a users' most recent tweets.

In [5]:
def max_limit(limit):
    if limit <= 3200:
        return limit
    else:
        return 3200

In [6]:
public_tweets = api.home_timeline(count=50)

Define some functions to retrieve a search or tweets from the timeline:
Note that the search may wait until there is not rate limit due to 'wait_on_rate_limit'.

In [7]:
def tweet_query(api, query, lang="en", limit=LIMIT):
    tweets = []
    for tweet in tqdm(tw.Cursor(api.search, q=query, lang=lang).items(max_limit(limit))):
        tweets.append(tweet)

    return tweets

Similarly to retrieve from the timeline....

In [8]:
def getTimeline(api, limit=LIMIT, resultType="recent"):
    lim = max_limit(limit)
    try:
        tweets = []
        tweetsObj = tw.Cursor(api.home_timeline,
                result_type=resultType,
                exclude_replies = False).items(lim)

        pBar = tqdm(tweetsObj, ascii=True, total=lim, desc="Getting Tweets!")
        for cnt, tweet in enumerate(pBar):
            pBar.update(1)
            if not cnt < lim:
                break
            tweets.append(tweet)
    except tw.error.TweepError as et:
        print(et)
    except Exception as e:
        print(e)
    return tweets 

Extract some attributes from the tweets:

In [9]:
tweet_columns = ["screen_name", "location", "id", "source", "coordinates", "favorite_count", 
                 "favorited", "lang", "hashtags", "created_at", "text"]
def get_tweet_info(tweet_list):
    return [[tweet.user.screen_name, tweet.user.location, tweet.user.id, tweet.source, 
             tweet.coordinates, tweet.favorite_count, tweet.favorited, tweet.lang,  
             tweet.entities['hashtags'], tweet.created_at, tweet.text] 
            for tweet in tweet_list]

### Retrieve User Info

In [10]:
def make_random_id_numbers(list_size, max_id=3400000000):
    """
    Return a list of random ID's 
    """
    random_integers = [random.randint(0, max_id) for id in range(0, list_size)]
    return random_integers

In [11]:
database_of_users = []

In [12]:
for sample_number in trange(1, 10):
    numbers = make_random_id_numbers(100)
    
    for n in numbers:
        try:
            sample_user_info = api.get_user(n)
            database_of_users.append(sample_user_info)
        except tw.TweepError as err:
            # 50   -> user not found
            # 63   -> user suspended
            # None -> "Failed to send request. Max retries exceeded with url"
            if err.api_code not in [50, 63, None]:
                breakpoint()
                pass
    sleep(5)

100%|██████████| 9/9 [57:05<00:00, 337.43s/it]


Write the user information to the file...

In [13]:
create_header = False if os.path.exists(TWITER_USERS_FILE) else True

with open(TWITER_USERS_FILE, 'a') as f:
    csv_writer = csv.writer(f)
    columns = ['name', 'screen_name', 'id', 'lang', 'followers_count', 'location', 'created_at', 
              'statuses_count', 'friends_count', 'description']
    if create_header:
        csv_writer.writerow(columns)
    
    for user in database_of_users:
        if hasattr(user, 'status'):
            csv_writer.writerow([*[getattr(user, k) for k in columns[:-1]], 
                                 user.description.encode('utf-8')])

In [14]:
users_df = pd.read_csv(TWITER_USERS_FILE)
users_df.drop_duplicates(subset=['id'], inplace=True)
users_df.to_csv(TWITER_USERS_FILE, index=False)
print('Total Users: {}'.format(users_df.shape[0]))

Total Users: 1756


### Create a Pandas Dataframe From A List of Tweet Data
Once you have a list of items that you wish to work with, you can create a pandas dataframe.

In [15]:
tweet_search_results = tweet_query(api, SEARCH_WORDS)

1000it [00:52, 19.20it/s]


In [16]:
tweet_df = pd.DataFrame(data=get_tweet_info(tweet_search_results), 
                        columns=tweet_columns)

In [17]:
header = False if os.path.exists(TWEETS_OUTPUT) else True
    
with open(TWEETS_OUTPUT, 'a') as f:
    tweet_df.to_csv(f, header=header, index=False)

In [18]:
tweet_df.head(5)

,screen_name,location,id,source,coordinates,favorite_count,favorited,lang,hashtags,created_at,text
0,Bellesde_nuit,,1143451156821270528,Twitter for iPhone,None,0,False,en,"[{'text': 'shortfilm', 'indices': [127, 137]}]",2019-07-08 22:14:01,RT @RianneMartyn: Good production meeting toda...
1,Stanzie_Actress,"Bristol, England",1006849914825998336,Twitter for Android,None,0,False,en,"[{'text': 'modelling', 'indices': [14, 24]}, {...",2019-07-08 22:13:27,Available for #modelling and #castings for #fi...
2,octo47,"Seoul, Korea",226171123,Twitter Web Client,None,0,False,en,"[{'text': 'smile', 'indices': [27, 33]}, {'tex...",2019-07-08 22:13:15,RT @noveliciouss: Just one #smile immensely in...
3,letsveer,United States,752081253113880577,Twitter Web Client,None,0,False,en,[],2019-07-08 22:12:16,"As the article says, ""VR and Film to Change th..."
4,BigJohnkap,"Atlanta,Ga",602049304,Twitter for Android,None,0,False,en,"[{'text': 'white', 'indices': [22, 28]}, {'tex...",2019-07-08 22:12:16,I've been cast as the #white #trans #lgtbq #ge...


In [20]:
tweet_df = pd.read_csv(TWEETS_OUTPUT)

In [21]:
tweet_df.shape

(1000, 11)